In [1]:
# notes:

# inventory, item stock, prediction for inventory
# f.e. next month how we will sell from each item

# top most selling products
# top most high selling products

# Importing libraries and cleaned dataset

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from prophet import Prophet

from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import mean_absolute_error

from functions import prophet_model

#import holidays 

Importing plotly failed. Interactive plots will not work.


In [3]:
# Disabling a specific type of warning from Jupyter, that I know if mistaken for something else
pd.options.mode.chained_assignment = None 

**Original Dataset**:

Chen,Daqing. (2015). Online Retail. UCI Machine Learning Repository. https://archive.ics.uci.edu/dataset/352/online+retail

In [4]:
# Importing dataset, clean and filtered with clusters from the notebook 03_ML_clustering
# the filters on are: only actual products, only UK, dropping NaN customerID
file_path1 = '../data/data_cleaned.csv'
file_path2 = '../data/data_filtered_with_clusters.csv'

df = pd.read_csv(file_path1, dtype={'InvoiceNo': 'O','StockCode': 'O','Description': 'O',
                                   'Quantity': 'int64','UnitPrice': 'float64',
                                   'CustomerID': 'Int64','Country': 'O'}, parse_dates=['InvoiceDate'])
df_fltrd = pd.read_csv(file_path2, dtype={'InvoiceNo': 'O','StockCode': 'O','Description': 'O',
                                   'Quantity': 'int64','UnitPrice': 'float64',
                                   'CustomerID': 'Int64','Country': 'O'}, parse_dates=['InvoiceDate'])

In [5]:
# test
df = df[df.StockCode != 'OTHER']
df = df[df.Country == 'United Kingdom']
df.dropna(subset='CustomerID', inplace=True)
df.reset_index(inplace=True)

In [6]:
# Ading the total price column to df
df['TotalPrice'] = df.Quantity * df.UnitPrice

# Feature Engineering

### Clipping outliers

I know from the EDA, shown in the previous notebooks, **two outliers** with huge difference in scale. I am going to do scaling to a range, scaling them to highest next value (after these two).

In [7]:
# scaling within a range, updating the Quantity and accordingly the TotalPrice
df.loc[df.InvoiceNo.isin(['581483','541431']),'Quantity'] = df.Quantity.sort_values(ascending=False).values[2]
df.loc[df.InvoiceNo.isin(['581483','541431']),'TotalPrice'] = df.Quantity * df.UnitPrice

df_fltrd.loc[df_fltrd.InvoiceNo.isin(['581483','541431']),'Quantity'] = df_fltrd.Quantity.sort_values(ascending=False).values[2]
df_fltrd.loc[df_fltrd.InvoiceNo.isin(['581483','541431']),'TotalPrice'] = df_fltrd.Quantity * df_fltrd.UnitPrice

### Date column

Adding new column for the date without time.

In [8]:
# New date column without time
df['InvoiceDate_noTime'] = df.InvoiceDate.dt.normalize()
df_fltrd['InvoiceDate_noTime'] = df_fltrd.InvoiceDate.dt.normalize()

### Removing features that will not be used

I will be using StockCode, Quantity and date for my model. 

In [9]:
df_stock = df[['StockCode','Quantity','InvoiceDate_noTime']].rename(columns={'InvoiceDate_noTime':'InvoiceDate'})
df_stock.head()

,StockCode,Quantity,InvoiceDate
0,85123A,6,2010-12-01
1,71053,6,2010-12-01
2,84406B,8,2010-12-01
3,84029G,6,2010-12-01
4,84029E,6,2010-12-01


In [10]:
df_fltrd_stock = df_fltrd[['StockCode','Quantity','InvoiceDate_noTime','Label']].rename(columns={'InvoiceDate_noTime':'InvoiceDate'})
df_fltrd_stock

,StockCode,Quantity,InvoiceDate,Label
0,85123A,6,2010-12-01,2
1,71053,6,2010-12-01,2
2,84406B,8,2010-12-01,2
3,84029G,6,2010-12-01,2
4,84029E,6,2010-12-01,2
...,...,...,...,...
348882,22466,12,2011-12-09,1
348883,22061,8,2011-12-09,2
348884,23275,24,2011-12-09,2
348885,21217,24,2011-12-09,2


# Splitting dataset

In [11]:
df_fltrd_stock.Label.unique()

array([2, 1, 0, 3])

In [12]:
df_fltrd_stock.Label.info()

<class 'pandas.core.series.Series'>
RangeIndex: 348887 entries, 0 to 348886
Series name: Label
Non-Null Count   Dtype
--------------   -----
348887 non-null  int64
dtypes: int64(1)
memory usage: 2.7 MB


In [13]:
df_fltrd_stock_0 = df_fltrd_stock[df_fltrd_stock.Label == 0].reset_index(drop=True)
df_fltrd_stock_1 = df_fltrd_stock[df_fltrd_stock.Label == 1].reset_index(drop=True)
df_fltrd_stock_2 = df_fltrd_stock[df_fltrd_stock.Label == 2].reset_index(drop=True)
df_fltrd_stock_3 = df_fltrd_stock[df_fltrd_stock.Label == 3].reset_index(drop=True)

# Top selling products

In [14]:
# the top 5 selling products
#products_top = df.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False).head(5).index.tolist()
#products_top

In [15]:
# the top 5 selling products
products_top = df_fltrd.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False).head(5).index.tolist()
products_top

['84077', '22197', '85099B', '85123A', '84879']

## Preparing dataframes for model

In [16]:
def df_prep_all_prod(df_2,clipping = False):
    ''' for building a more generic model, for all the products that will be sold.
    Mainly for research purpose, cannot be used for inventory.'''
    df = df_2.copy()

    # Dropping StockCode, will not be used for this model
    df.drop(columns='StockCode', inplace=True)

    # Adding rows for all dates, even if there is no data
    # Define the known start and end dates of the dataset
    start_date = '2010-12-01'
    end_date = '2011-12-09'
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    all_dates_df = pd.DataFrame({'InvoiceDate': all_dates})

    # merging the date dataframe with the main dataframe
    merged_df = all_dates_df.merge(df, on='InvoiceDate', how='left').fillna(0)

    # and then grouping to make sure, I have only ont line per day
    merged_df = merged_df.groupby('InvoiceDate')['Quantity'].sum().reset_index()

    for i in range(merged_df.shape[1]):
        if merged_df.iloc[:,i].dtypes =='<M8[ns]':
            merged_df.rename(columns={merged_df.columns[i]: "ds"}, inplace=True)
        else:
            merged_df.rename(columns={merged_df.columns[i]: "y"}, inplace=True)

    merged_df = merged_df[['ds', 'y']] 
    
    if clipping == True:
        tq_UpperLimit = (merged_df.y.mean() + merged_df.y.std()*3).round(0).astype(int)
        merged_df_clip = merged_df.copy()
        merged_df_clip.loc[merged_df_clip.y > tq_UpperLimit , 'y'] = tq_UpperLimit        
        return merged_df_clip
    else:
        return merged_df


In [17]:
def df_prep_top_prod(df_2,products_top, product_i,filtering=False, clipping = False):
    df = df_2.copy()
    
    if filtering==True:
        df = df[df.StockCode == products_top[product_i]].drop(columns='StockCode').reset_index(drop=True)
    else:
        df.drop(columns='StockCode', inplace=True)

    
    # Adding rows for all dates, even if there is no data
    # Define the known start and end dates of the dataset
    start_date = '2010-12-01'
    end_date = '2011-12-09'
    all_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    all_dates_df = pd.DataFrame({'InvoiceDate': all_dates})

    # merging the date dataframe with the main dataframe
    merged_df = all_dates_df.merge(df, on='InvoiceDate', how='left').fillna(0)

    # and then grouping to make sure, I have only ont line per day
    merged_df = merged_df.groupby('InvoiceDate')['Quantity'].sum().reset_index()
    

    for i in range(merged_df.shape[1]):
        if merged_df.iloc[:,i].dtypes =='<M8[ns]':
            merged_df.rename(columns={merged_df.columns[i]: "ds"}, inplace=True)
        else:
            merged_df.rename(columns={merged_df.columns[i]: "y"}, inplace=True)

    merged_df = merged_df[['ds', 'y']] 
    
    if clipping == True:
        tq_UpperLimit = (merged_df.y.mean() + merged_df.y.std()*3).round(0).astype(int)
        merged_df_clip = merged_df.copy()
        merged_df_clip.loc[merged_df_clip.y > tq_UpperLimit , 'y'] = tq_UpperLimit        
        return merged_df_clip
    else:
        return merged_df


In [18]:
df_stock_top_pr = df_prep_top_prod(df_stock, products_top, 0, True,False)
df_stock_top_pr

,ds,y
0,2010-12-01,0.0
1,2010-12-02,3264.0
2,2010-12-03,48.0
3,2010-12-04,0.0
4,2010-12-05,96.0
...,...,...
369,2011-12-05,205.0
370,2011-12-06,240.0
371,2011-12-07,192.0
372,2011-12-08,336.0


In [19]:
df_stock_top_pr_clip = df_prep_top_prod(df_stock, products_top, 0,True, True)

In [20]:
df_fltrd_stock_0_top_pr_clip = df_prep_top_prod(df_fltrd_stock_0.drop(columns='Label'), products_top, 0, True,True)
df_fltrd_stock_1_top_pr_clip = df_prep_top_prod(df_fltrd_stock_1.drop(columns='Label'), products_top, 0, True,True)
df_fltrd_stock_2_top_pr_clip = df_prep_top_prod(df_fltrd_stock_2.drop(columns='Label'), products_top, 0, True,True)
df_fltrd_stock_3_top_pr_clip = df_prep_top_prod(df_fltrd_stock_3.drop(columns='Label'), products_top, 0, True,True)

In [21]:
df_fltrd_stock_prep = df_prep_all_prod(df_fltrd_stock[['StockCode','Quantity','InvoiceDate']], False)
df_fltrd_stock_clip = df_prep_all_prod(df_fltrd_stock[['StockCode','Quantity','InvoiceDate']], True)

In [22]:
df_stock_prep = df_prep_all_prod(df_stock, False)
df_stock_clip = df_prep_all_prod(df_stock, True)

# Prophet model

https://facebook.github.io/prophet/docs/quick_start.html

### general sales

In [23]:
# For all products, not filtered model
forecast = prophet_model(df_stock_prep, 73)

19:05:06 - cmdstanpy - INFO - Chain [1] start processing
19:05:06 - cmdstanpy - INFO - Chain [1] done processing


Root Mean Squared Error:  7776.063017888674
Mean Absolute Error:  6170.024600219451


In [24]:
# For all products, not filtered model, outliers clipped
forecast  = prophet_model(df_stock_clip, 73)

19:05:06 - cmdstanpy - INFO - Chain [1] start processing
19:05:06 - cmdstanpy - INFO - Chain [1] done processing


Root Mean Squared Error:  7843.1630102082645
Mean Absolute Error:  6218.881621879992


In [25]:
forecast = prophet_model(df_fltrd_stock_prep, 73)

19:05:06 - cmdstanpy - INFO - Chain [1] start processing
19:05:06 - cmdstanpy - INFO - Chain [1] done processing


Root Mean Squared Error:  7776.063017888674
Mean Absolute Error:  6170.024600219451


In [26]:
forecast = prophet_model(df_fltrd_stock_clip, 73)

19:05:06 - cmdstanpy - INFO - Chain [1] start processing
19:05:06 - cmdstanpy - INFO - Chain [1] done processing


Root Mean Squared Error:  7843.1630102082645
Mean Absolute Error:  6218.881621879992


### per product

In [27]:
# 365-(365*0.8)  # 73.0
# Calling function for plrophet model, with defining the number of days for the test set.
forecast = prophet_model(df_stock_top_pr, 73)
# not good

19:05:06 - cmdstanpy - INFO - Chain [1] start processing
19:05:06 - cmdstanpy - INFO - Chain [1] done processing


Root Mean Squared Error:  556.2757083286609
Mean Absolute Error:  152.08179586275608


In [28]:
# trying the model where outliers were clipped
forecast = prophet_model(df_stock_top_pr_clip, 73)

# better, but not good yet

19:05:06 - cmdstanpy - INFO - Chain [1] start processing
19:05:06 - cmdstanpy - INFO - Chain [1] done processing


Root Mean Squared Error:  167.63857502240813
Mean Absolute Error:  88.57629350800431


In [45]:
# testing with less days for the test model
forecast = prophet_model(df_stock_top_pr_clip, 50)
# not better, also very small test sample

19:07:24 - cmdstanpy - INFO - Chain [1] start processing
19:07:24 - cmdstanpy - INFO - Chain [1] done processing


Root Mean Squared Error:  194.53711472109725
Mean Absolute Error:  98.12634202344665


### per product per cluster

In [42]:
forecast_p0_l0,rmse_p0_l0 = prophet_model(df_fltrd_stock_0_top_pr_clip, 50)

19:05:44 - cmdstanpy - INFO - Chain [1] start processing
19:05:44 - cmdstanpy - INFO - Chain [1] done processing


Root Mean Squared Error:  26.550302752060936
Mean Absolute Error:  17.58670865191472


In [31]:
forecast_p0_l1,rmse_p0_l1 = prophet_model(df_fltrd_stock_1_top_pr_clip, 50)

19:05:06 - cmdstanpy - INFO - Chain [1] start processing
19:05:06 - cmdstanpy - INFO - Chain [1] done processing


Root Mean Squared Error:  57.73997577956072
Mean Absolute Error:  43.991045630955576


In [32]:
forecast_p0_l2,rmse_p0_l2 = prophet_model(df_fltrd_stock_2_top_pr_clip, 50)

19:05:06 - cmdstanpy - INFO - Chain [1] start processing
19:05:07 - cmdstanpy - INFO - Chain [1] done processing


Root Mean Squared Error:  68.61645046603645
Mean Absolute Error:  43.44641525337744


In [33]:
forecast_p0_l3,rmse_p0_l3 = prophet_model(df_fltrd_stock_3_top_pr_clip, 50)

19:05:07 - cmdstanpy - INFO - Chain [1] start processing
19:05:07 - cmdstanpy - INFO - Chain [1] done processing


Root Mean Squared Error:  164.9060184969039
Mean Absolute Error:  47.63167720852738


In [34]:
label_percentages = [round((count / df_fltrd_stock.shape[0]) * 100,2) for count in [df_fltrd_stock[df_fltrd_stock.Label == i].shape[0] for i in range(4)]]
label_percentages

[18.97, 47.2, 23.95, 9.88]

In [35]:
prop0 = rmse_p0_l0*label_percentages[0]*0.01
prop1 = rmse_p0_l1*label_percentages[1]*0.01
prop2 = rmse_p0_l2*label_percentages[2]*0.01
prop3 = rmse_p0_l3*label_percentages[3]*0.01

In [36]:
# Calculating the total prediction, based on the proportion of the cluster
pred_p0_total = prop0 + prop1 + prop2 + prop3
pred_p0_total

64.5616572662076